In [28]:
# Set up.
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import matplotlib as plt
import numpy as np
import pandas as pd
import seaborn

import PyPDF2

In [2]:
bill = PyPDF2.PdfFileReader(open('PastBills.pdf', 'rb'))

In [3]:
bill.numPages

25

In [5]:
usage_page_1 = bill.getPage(3)

In [7]:
text_page_1 = usage_page_1.extractText()

In [8]:
type(text_page_1)

str

In [9]:
text_page_1.split('\n')

['Statement for',
 'Account number',
 'Bill close date',
 'KURT MAURER',
 '834541363',
 'Aug 18, 2017',
 'Type: ',
 '(A) Call Waiting ',
 '(B) Call Forward ',
 '(C) Conference Call ',
 '(D) Duplicate ',
 '(E) Data/Fax ',
 '(F) Mobile2Mobile ',
 '(G) Voicemail ',
 '(H) Free Calls ',
 '(I) Intl Disc Call ',
 '(J) Intl Disc',
 'Call to Mobile ',
 '(K) WPS Call ',
 '(M) AnyMobile ',
 '(R) Roaming ',
 '(T) T-Mobile Number ',
 '(V) myFaves Call ',
 '(W) Wi-Fi Call ',
 '(X) T-Mobile @Home Call ',
 '(Y) Subscription ',
 '(Z) One-',
 'time Purchase',
 'Page ',
 '4',
 ' of ',
 '25',
 'Usage details',
 '(334) 524-9020',
 'Talk',
 'The date and time corresponds to the local time where the mobile was located.',
 'Date and time',
 'Number',
 'Description',
 'Type',
 'Min',
 'Amount',
 '07/20/17, 4:36 PM',
 '(317) 432-3506',
 'to INDIANAPLS/IN',
 '-',
 '52',
 '-',
 '07/20/17, 8:55 PM',
 '(334) 703-1602',
 'to AUBURN/AL',
 '-',
 '1',
 '-',
 '07/20/17, 8:56 PM',
 '(334) 703-1602',
 'to AUBURN/AL',
 '-'

In [10]:
split_text_1 = text_page_1.split('\n')

In [11]:
split_text_1.index('Date and time')

36

In [12]:
split_text_1.index('07/20/17, 4:36 PM')

42

In [40]:
split_text_1.remove('')

In [41]:
bill_dict = {}
for column in split_text_1[36:42]:
    column_index = split_text_1.index(column)
    bill_dict[column] = split_text_1[column_index + 6::6]

In [42]:
bill_dict

{'Amount': ['-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-'],
 'Date and time': ['07/20/17, 4:36 PM',
  '07/20/17, 8:55 PM',
  '07/20/17, 8:56 PM',
  '07/21/17, 12:41 PM',
  '07/21/17, 6:47 PM',
  '07/22/17, 9:58 AM',
  '07/24/17, 12:21 PM',
  '07/24/17, 6:40 PM',
  '07/25/17, 7:47 PM',
  '07/26/17, 6:11 PM',
  '07/26/17, 7:20 PM',
  '07/26/17, 10:09 PM',
  '07/26/17, 10:17 PM',
  '07/27/17, 6:43 PM',
  '07/27/17, 10:07 PM',
  '07/28/17, 6:15 PM',
  '07/29/17, 12:25 PM',
  '07/31/17, 11:03 AM',
  '07/31/17, 11:14 AM',
  '07/31/17, 12:07 PM',
  '07/31/17, 3:55 PM',
  '08/01/17, 8:13 AM',
  '08/02/17, 5:06 PM',
  '08/0

In [39]:
len(bill_dict['Date and time'])

62

In [43]:
for key, value in bill_dict.items():
    print(len(value))

61
61
61
61
61
61


In [45]:
bill_df = pd.DataFrame(bill_dict)

In [46]:
bill_df

,Amount,Date and time,Description,Min,Number,Type
0,-,"07/20/17, 4:36 PM",to INDIANAPLS/IN,52,(317) 432-3506,-
1,-,"07/20/17, 8:55 PM",to AUBURN/AL,1,(334) 703-1602,-
2,-,"07/20/17, 8:56 PM",to AUBURN/AL,1,(334) 703-1602,-
3,-,"07/21/17, 12:41 PM",to AUBURN/AL,19,(334) 728-0615,(F)
4,-,"07/21/17, 6:47 PM",to AUBURN/AL,1,(334) 703-1602,-
5,-,"07/22/17, 9:58 AM",to AUBURN/AL,3,(334) 703-1602,-
6,-,"07/24/17, 12:21 PM",to AUBURN/AL,6,(334) 703-1602,-
7,-,"07/24/17, 6:40 PM",to AUBURN/AL,7,(334) 703-1602,-
8,-,"07/25/17, 7:47 PM",to AUBURN/AL,1,(334) 703-1602,-
9,-,"07/26/17, 6:11 PM",to CARBONDALE/PA,35,(570) 267-5289,-


In [61]:
usage_page_2 = bill.getPage(4)
text_page_2 = usage_page_2.extractText()
split_text_page_2 = text_page_2.split('\n')
while '' in split_text_page_2:
    split_text_page_2.remove('')

In [64]:
split_text_page_2[37::].index('Date and time')

190

In [65]:
split_text_1.count('Date and time')

1

In [66]:
split_text_page_2

['Statement for',
 'Account number',
 'Bill close date',
 'KURT MAURER',
 '834541363',
 'Aug 18, 2017',
 'Continued... ',
 '(334) 524-9020',
 'Type: ',
 '(A) Call Waiting ',
 '(B) Call Forward ',
 '(C) Conference Call ',
 '(D) Duplicate ',
 '(E) Data/Fax ',
 '(F) Mobile2Mobile ',
 '(G) Voicemail ',
 '(H) Free Calls ',
 '(I) Intl Disc Call ',
 '(J) Intl Disc',
 'Call to Mobile ',
 '(K) WPS Call ',
 '(M) AnyMobile ',
 '(R) Roaming ',
 '(T) T-Mobile Number ',
 '(V) myFaves Call ',
 '(W) Wi-Fi Call ',
 '(X) T-Mobile @Home Call ',
 '(Y) Subscription ',
 '(Z) One-',
 'time Purchase',
 'Page ',
 '5',
 ' of ',
 '25',
 'Talk',
 'The date and time corresponds to the local time where the mobile was located.',
 'Date and time',
 'Number',
 'Description',
 'Type',
 'Min',
 'Amount',
 '08/11/17, 12:14 PM',
 '(334) 728-0615',
 'Incoming',
 '(F)',
 '3',
 '-',
 '08/11/17, 4:12 PM',
 '(334) 559-0212',
 'Incoming',
 '(F)',
 '1',
 '-',
 '08/11/17, 6:40 PM',
 '(334) 728-0615',
 'Incoming',
 '(F)',
 '6',
 '